# COVID cases by community since June 15

### Import Python tools and Jupyter configuration

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import datetime as dt
import matplotlib.pyplot as plt
import altair as alt
import numpy as np

/Users/mhustiles/.local/share/virtualenvs/notebooks-xu8v5Vvv/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [3]:
pd.options.display.max_columns = 100
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

### Read the raw data

In [4]:
src = pd.read_json("../../coronavirus-tracker/_data/places/latest/all_.json")

In [5]:
df = src[src["date"] >= "2021/06/15"]

In [6]:
df.head()

,name,date,county,fips,slug,confirmed_cases_note,confirmed_cases,lat,lon,population,zcta_id,id,new_confirmed_cases,new_confirmed_cases_seven_day_total,new_confirmed_cases_seven_day_average,new_confirmed_cases_fourteen_day_total,new_confirmed_cases_fourteen_day_average,confirmed_cases_per_100k,new_confirmed_cases_per_100k,new_confirmed_cases_seven_day_per_100k,new_confirmed_cases_fourteen_day_per_100k
473,94501: Alameda,2021-06-15,Alameda,1,94501-alameda,None,2206,37.774606,-122.274583,62826.0,94501.0,94501,0,10.0,1.428571,15.0,1.071429,3511.285137,0.000000,15.916977,23.875466
474,94501: Alameda,2021-06-16,Alameda,1,94501-alameda,None,2206,37.774606,-122.274583,62826.0,94501.0,94501,0,10.0,1.428571,15.0,1.071429,3511.285137,0.000000,15.916977,23.875466
475,94501: Alameda,2021-06-17,Alameda,1,94501-alameda,None,2209,37.774606,-122.274583,62826.0,94501.0,94501,3,13.0,1.857143,15.0,1.071429,3516.060230,4.775093,20.692070,23.875466
476,94501: Alameda,2021-06-18,Alameda,1,94501-alameda,None,2209,37.774606,-122.274583,62826.0,94501.0,94501,0,6.0,0.857143,15.0,1.071429,3516.060230,0.000000,9.550186,23.875466
477,94501: Alameda,2021-06-19,Alameda,1,94501-alameda,None,2210,37.774606,-122.274583,62826.0,94501.0,94501,1,7.0,1.000000,16.0,1.142857,3517.651928,1.591698,11.141884,25.467163


---

### Calculate 38-day change to seven-day case rate in larger LA communities

In [7]:
trimmed_df_la = df[
    (df["county"] == "Los Angeles")
    & (df["population"] > 1000)
    & (df["new_confirmed_cases_seven_day_per_100k"] > 0)
    & (~df["name"].str.contains(":"))
].copy()

In [8]:
trimmed_df_la.columns

Index(['name', 'date', 'county', 'fips', 'slug', 'confirmed_cases_note',
       'confirmed_cases', 'lat', 'lon', 'population', 'zcta_id', 'id',
       'new_confirmed_cases', 'new_confirmed_cases_seven_day_total',
       'new_confirmed_cases_seven_day_average',
       'new_confirmed_cases_fourteen_day_total',
       'new_confirmed_cases_fourteen_day_average', 'confirmed_cases_per_100k',
       'new_confirmed_cases_per_100k',
       'new_confirmed_cases_seven_day_per_100k',
       'new_confirmed_cases_fourteen_day_per_100k'],
      dtype='object')

In [9]:
trimmed_df_la["new_confirmed_cases_month_change"] = (
    trimmed_df_la.new_confirmed_cases_seven_day_per_100k.pct_change(14)
    .replace([np.inf, -np.inf], -1)
    .round(2)
) * 100

In [10]:
slim_df_la = trimmed_df_la[
    [
        "name",
        "date",
        "county",
        "fips",
        "slug",
        "lat",
        "lon",
        "population",
        "zcta_id",
        "id",
        "new_confirmed_cases_seven_day_per_100k",
        "new_confirmed_cases_month_change",
    ]
]

In [11]:
slim_latest_df_la = slim_df_la[slim_df_la["date"] == slim_df_la["date"].max()]

In [12]:
slim_latest_df_la.sort_values("new_confirmed_cases_month_change", ascending=False).head(
    30
)

,name,date,county,fips,slug,lat,lon,population,zcta_id,id,new_confirmed_cases_seven_day_per_100k,new_confirmed_cases_month_change
246162,Stevenson Ranch,2021-07-23,Los Angeles,37,unincorporated-stevenson-ranch,34.376825,-118.607266,20966.0,NaN,Unincorporated - Stevenson Ranch,171.706573,3500.0
223887,Rancho Park,2021-07-23,Los Angeles,37,los-angeles-rancho-park,34.036458,-118.429114,6560.0,NaN,Los Angeles - Rancho Park,213.414634,1300.0
134292,Bassett,2021-07-23,Los Angeles,37,unincorporated-bassett,34.057144,-117.977406,14817.0,NaN,Unincorporated - Bassett,94.486063,1300.0
274377,Walnut Park,2021-07-23,Los Angeles,37,unincorporated-walnut-park,33.968237,-118.221950,16143.0,NaN,Unincorporated - Walnut Park,74.335625,1100.0
186267,La Mirada,2021-07-23,Los Angeles,37,city-of-la-mirada,33.902510,-118.009320,49599.0,NaN,City of La Mirada,118.954011,1080.0
185277,La Crescenta-Montrose,2021-07-23,Los Angeles,37,unincorporated-la-crescenta-montrose,34.232152,-118.235266,19801.0,NaN,Unincorporated - La Crescenta-Montrose,116.155750,1050.0
187752,La Verne,2021-07-23,Los Angeles,37,city-of-la-verne,34.120692,-117.770312,33282.0,NaN,City of La Verne,93.143441,933.0
128847,Arcadia,2021-07-23,Los Angeles,37,city-of-arcadia,34.134186,-118.037297,57754.0,NaN,City of Arcadia,124.666690,929.0
237252,Santa Monica Mountains,2021-07-23,Los Angeles,37,unincorporated-santa-monica-mountains,34.087782,-118.740912,18621.0,NaN,Unincorporated - Santa Monica Mountains,161.108426,900.0
220422,Playa Del Rey,2021-07-23,Los Angeles,37,los-angeles-playa-del-rey,33.961018,-118.446368,3196.0,NaN,Los Angeles - Playa Del Rey,312.891114,900.0


In [13]:
slim_latest_df_la_gdf = gpd.GeoDataFrame(
    slim_latest_df_la,
    geometry=gpd.points_from_xy(slim_latest_df_la.lon, slim_latest_df_la.lat),
)

In [14]:
slim_latest_df_la_gdf.to_file("output/latest_la_change.geojson", driver="GeoJSON")